# Análise de Dados

Nesse notebook será feita uma análise dos dados das bases já trabalhadas.

In [1]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import ast
import json
import seaborn as sns
from IPython.display import Image, HTML

  
#Função para análise de valores ausentes
def missingValuesSnapshot(df):
    missing_df = df.isnull().sum(axis=0).reset_index()
    missing_df.columns = ['column_name', 'missing_count']
    missing_df['filling_factor'] = (df.shape[0] 
                                - missing_df['missing_count']) / df.shape[0] * 100
    missing_df.sort_values('filling_factor', inplace=True)
    
    print(missing_df)
    
#Leitura de arquivos

#Golden Globe
gg_path = "../bases_tratadas/gg_df.csv"
#codificação  "ISO-8859-1"
gg_df = pd.read_csv(gg_path, encoding = "ISO-8859-1")

#EMMY 
emmy_path = "../bases_tratadas/emmy_df.csv"
emmy_df = pd.read_csv(emmy_path)

# Nomeações e premiações por série
prizes_nominees_path = '../bases_tratadas/prizes_nominees_df.csv'
prizes_nominees_df = pd.read_csv(prizes_nominees_path)

# Séries com dados TMDB e IMDB - união
series_path = "../bases_tratadas/series_imdb_tmdb_df.csv"
series_df = pd.read_csv(series_path)

ModuleNotFoundError: No module named 'wordcloud'

## Dataset de Séries

### Análise de métricas - votos e popularidade

Será feito um estudo sobre os dados de métricas obtidas por meio da interação dos usuários com as bases do TMDB e IMDB: popularidade, número de votos e média da nota:

In [ ]:
def clean_numeric(x):
    try:
        return float(x)
    except:
        return np.nan


In [ ]:
series_df['popularity'] = series_df['popularity'].apply(clean_numeric).astype('float')
series_df['numVotesTMDB'] = series_df['numVotesTMDB'].apply(clean_numeric).astype('float')
series_df['averageRatingTMDB'] = series_df['averageRatingTMDB'].apply(clean_numeric).astype('float')
series_df['numVotesIMDB'] = series_df['numVotesIMDB'].apply(clean_numeric).astype('float')
series_df['averageRatingIMDB'] = series_df['averageRatingIMDB'].apply(clean_numeric).astype('float')

Vamos analisar o resumo estatístico de cada um desses atributos:

#### Popularidade
É uma métrica do TMDB baseada em vários quesitos: nº de votos no dia, nº de visualizações no dia, nº de usuários que 'favoritaram' a série naquele dia, nº de usuários que adicionaram a série à "watchlist" no dia, data de lançamento do próximo ou último episódio, número total de votos, pontuação em dias anteriores.
    
Vemos que é uma métrica baseada na popularidade do momento atual, pois leva em consideração muitos quesitos referentes ao dia, mais do que quesitos acumulados ao longo do tempo. 

In [ ]:
series_df['popularity'].describe()

In [ ]:
sns.distplot(series_df['popularity'])
plt.show()

A pontuação de popularidade parece ser uma quantidade extremamente distorcida, com uma média de apenas 0,57, mas valores máximos atingindo 318.6. No entanto, como pode ser visto no gráfico de distribuição, quase todos os filmes têm uma pontuação de popularidade menor que 1 (o 75º percentil está em 0,6). 
Somente registros provenientes da base do TMDB apresentam valores não zerados de popularidade.
Vamos ver as séries mais populares:

In [ ]:
# Alterando o caminho do poster para incluir tags HTML
series_df['poster_path'] = "<img src='" + series_df['poster_path'] + "' style='height:100px;'>"

pop_top = series_df[['poster_path', 'name', 'popularity', 'startSeriesYear']].sort_values('popularity', ascending=False).head(10)
pd.set_option('display.max_colwidth', 100)
HTML(pop_top.to_html(escape=False))

Dos top 10 em popularidade temos 4 com a temática super herói: The Flash, em primeiro, Arrow, em terceiro lugar, The Boys, em sétimo lugar e Marvel's Agents of S.H.I.E.L.D. em décimo. Temos duas séries de desenho: Family Guy e The Simpsons.

#### Número de Votos - TMDB e IMDB

In [ ]:
series_df['numVotesIMDB'].describe()

In [ ]:
sns.distplot(series_df['numVotesIMDB'])

In [ ]:
series_df['numVotesTMDB'].describe()

In [ ]:
sns.distplot(series_df['numVotesTMDB'])

Observa-se que a quantidade de votos no IMDB é muitas ordens de grandeza maior que a quantidade de votos no TMDB. A base do IMDB é bem mais popular e suas votações bem mais expressivas. Podemos ver também que no TMDB (até o 3º quartil), mas também no IMDB (até o 2º), grande número de séries NÃO receberam votos. Lembrando que esse dataset é a junção (via OUTER join) das duas bases e que essas métricas são específicas de cada base.
Vamos verificar quais são as séries mais votadas nos sites:

In [ ]:
vote_top = series_df[['poster_path', 'name', 'numVotesIMDB', 'startSeriesYear']].sort_values('numVotesIMDB', ascending=False).head(10)
pd.set_option('display.max_colwidth', 100)
HTML(vote_top.to_html(escape=False))

In [ ]:
vote_top = series_df[['poster_path', 'name', 'numVotesTMDB', 'startSeriesYear']].sort_values('numVotesTMDB', ascending=False).head(10)
pd.set_option('display.max_colwidth', 100)
HTML(vote_top.to_html(escape=False))

5 dos top 10 são coincidentes nas duas bases. Game Of Thrones é o mais votado em ambas.

#### Nota média - TMDB e IMDB

In [ ]:
series_df['averageRatingIMDB'].describe()

In [ ]:
sns.distplot(series_df['averageRatingIMDB'])

In [ ]:
series_df['averageRatingTMDB'].describe()

In [ ]:
sns.distplot(series_df['averageRatingTMDB'])

A classificação média dos sites também é baixa e mais alta para o IMDB (0,63 para TMDB e 2.02 no IMDB) pelo motivo já citado anteriormente: a junção das bases ter sido feita no modo OUTER join para trabalharmos com maior número de dados fez com que essas métricas específicas de cada base apresentassem muitos zeros.

Ao verificar quais são as 10 séries mais bem avaliadas pela crítica restringiremos a avaliação às séries que tiveram um número mínimo de votos: 1000 no caso do TMDB e 10000 no caso do IMDB, porque ter uma média alta com poucos avaliadores é fácil. 

In [ ]:
avg_top = series_df[series_df['numVotesIMDB'] > 10000][['poster_path', 'name', 'averageRatingIMDB', 'numVotesIMDB' ,'averageRatingTMDB', 'numVotesTMDB','original_language','startSeriesYear']].sort_values('averageRatingIMDB', ascending=False).head(10)
pd.set_option('display.max_colwidth', 100)
HTML(avg_top.to_html(escape=False))

In [ ]:
avg_top = series_df[series_df['numVotesTMDB'] > 1000][['poster_path', 'name', 'averageRatingTMDB', 'numVotesTMDB' ,'averageRatingIMDB', 'numVotesIMDB','original_language','startSeriesYear']].sort_values('averageRatingTMDB', ascending=False).head(10)
pd.set_option('display.max_colwidth', 100)
HTML(avg_top.to_html(escape=False))

Nas tabelas acima as séries foram ranqueadas primeiro pela nota média no IMDB e depois pela nota média do TMDB. Em ambas as tabelas mostramos informações de métricas dos dois sites para comparação. 

Uma descoberta interessante: observamos que para 4 séries do top 10 nota média do IMDB (com no mínimo 10.000 votos no site) temos uma votação insignificante no site do TMDB. Essas 4 séries (The Ants ou Koombiyo, The Filthy Frank Show,  College Romance e Yeh Meri Family) são asiáticas, duas indianas, o primeiro lugar é do Sri Lanka e o segundo lugar é uma série de um canal de um rapaz japonês no Youtube, algo que não havia sido previsto no início desse trabalho. Diante dessa observação podemos ponderar que o IMDB é uma base divulgada mundialmente, apresentando conteúdo e avaliações que ultrapassam o comum americano e englobam produções extremamente independentes e variadas, tais quais as presentes nos canais do Youtube. Já o TMDB é uma base mais focada em conteúdo americano e produtoras tradicionais.

### Análise de incidências

Primeiro vamos analisar as ocorrências nas colunas que informam gênero, país de origem, línguas, tempo por episódio e temporadas:

#### Gêneros

In [ ]:
def countDifferentItemsFromColumnList(df, columnName):
    df[columnName] = df[columnName].fillna('[]').apply(ast.literal_eval)
    s = df.apply(lambda x: pd.Series(x[columnName]),axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'occurrences'
    gen_df = series_df.drop(columnName, axis=1).join(s)
    print("#### " + columnName + "  ####");
    print("Number of items: " + str(gen_df['occurrences'].value_counts().shape[0]));
    pop_gen = pd.DataFrame(gen_df['occurrences'].value_counts()).reset_index()
    pop_gen.columns = ['occurrences', 'series']
    print("List of occurrences: ")
    print(pop_gen)
    print("Bar chart: ")
    plt.figure(figsize=(18,8))
    sns.barplot(x='occurrences', y='series', data=pop_gen.head(15))
    plt.show()

countDifferentItemsFromColumnList(series_df.copy(), 'genres')


TMDB e IMDB definem 38 denominações diferentes de gêneros. A maior incidência com considerável distância dos demais gêneros é de dramas e comédias. Não sem propósito, nas premiações temos várias categorias exclusivas para premiar séries desses dois gêneros.

Vamos aproveitar o momento para tratar os dados de gênero criando os atributos ehComedia, ehDrama e ehDocumentario para substituir o atributo 'genres' e diminuindo os valores ausentes do dataframe:

In [ ]:
#Função para informar se dado value está denstro dos valores de lista
def isValuesInStrList(strList, value):
    if isinstance(strList, float):
        return False
    else:
        strList = ast.literal_eval(strList)
        if isinstance(strList, list):
            for s in strList:
                if(s == value):
                    return True
            return False
        else:
            return False
        
#- para substituir o atributo genres:  ehComedia, ehDrama e ehDocumentario
series_df['ehComedia'] = series_df['genres'].apply(isValuesInStrList, args=('Comedy',))
series_df['ehDrama'] = series_df['genres'].apply(isValuesInStrList, args = ('Drama',))
series_df['ehDocumentario'] = series_df['genres'].apply(isValuesInStrList, args = ('Documentary',))
series_df = series_df.drop('genres', axis=1)
missingValuesSnapshot(series_df)

#### País de Origem

In [ ]:
countDifferentItemsFromColumnList(series_df.copy(), 'origin_country')

130 países de origem diferentes. Maiores representantes EUA, Grão Bretanha e Japão. O Brasil está em 17º lugar, com 494 séries.

Levantamos aqui uma hipótese: o número expresivo de séries japonesas tem como 'network' o YouTube? Vamos atualizar o notebook  4 para termos um atributo ehYouTube no dataframe e analisar a questão abaixo:

In [ ]:
temp_df = series_df[series_df['origin_country'].apply(isValuesInStrList, args=('JP',))]
print("Número de registros com país de origem Japão: " + str(temp_df.shape[0]));
temp_df = temp_df[temp_df['ehYouTube']];
print("Número de registros com país de origem Japão com network Youtube: " + str(temp_df.shape[0]));

Nossa hipótese não se comprovou...rs

#### Nº de episódios e Nº de Temporadas

Esses dois atributos são relacionados: quanto mais temporadas, mais episódios: existe uma correlação de 0,43. O fato de se ter mais temporadas apresenta um diferencial, pois uma temporada representa um novo ciclo contratual da série, o que é um indicativo de que a série está indo bem, é algo bem significativo. Vejamos o resumo estatístico desses dois atributos:

In [ ]:
series_df['number_of_episodes'].corr(series_df['number_of_seasons'])

In [ ]:
series_df['number_of_episodes'].value_counts().head(10)

In [ ]:
series_df[series_df['number_of_episodes'] > 0][['number_of_episodes', 'name', 'startSeriesYear']].sort_values(by = ['number_of_episodes', 'startSeriesYear'], ascending=False).head(10)

Pelo resumo de ocorrências da coluna 'number_of_episodes' vemos que a muitas séries foi atribuído o valor zero de episódios. Zero é o segundo valor mais frequente, sendo o primeiro um valor não inteiro 20.847526 que é o resultado da estratégia de preenchimento de campos nulos que foi aplicada no notebook 4.

Os primeiros resultados de séries com maior número de episódios são programas diários. Temos também novelas e shows de perguntas e respostas.

In [ ]:
series_df['number_of_seasons'].value_counts().head(10)

In [ ]:
series_df[series_df['number_of_seasons'] > 0][['number_of_seasons', 'name', 'startSeriesYear']].sort_values(by = ['number_of_seasons', 'startSeriesYear'], ascending=False).head(10)

Pelo resumo de ocorrências da coluna 'number_of_seasons' vemos que a muitas séries foi atribuído o valor zero ou um0 de temporadas. Zero é o terceiro valor mais frequente, sendo o segundo o valor um e o primeiro um valor não inteiro 1.31833 que é o resultado da estratégia de preenchimento de campos nulos que foi aplicada no notebook 4.

Exemplos das séries com mais temporadas:
Fitil é uma popular série de curtas-metragens satíricos / comédias de televisão soviética / russa. Alguns dos episódios foram dirigidos a crianças. Cada edição continha os poucos segmentos curtos: documentário, fictício e animado.

A Play School é um programa de televisão educacional premiado pela Australian Gold Logie para crianças produzido pela Australian Broadcasting Corporation. É o show infantil de maior duração na Austrália e o segundo show infantil de maior duração do mundo.


#### Tempo por episódio
Vamos analisar o tempo médio por episódio:

In [ ]:
series_df['episodeRuntimeSeries'].value_counts().head(10)

O tempo médio mais recorrente é um tempo fracionado, obtido  pela estratégia de preenchimento de campos nulos que foi aplicada no notebook 4. Em seguida temos séries de 30 e 60 minutos e depois de 45 e 25 minutos. Vamos ver se a duração média dos episódios das séries seguiu algum padrão ou apresenta alguma tendência ao longo do tempo:

In [ ]:
series_df['episodeRuntimeSeries'] = series_df['episodeRuntimeSeries'].astype('float')
plt.figure(figsize=(18,5))
year_runtime = series_df.groupby('startSeriesYear')['episodeRuntimeSeries'].mean()
plt.plot(year_runtime.index, year_runtime)
plt.xticks(np.arange(1874, 2024, 10.0))
plt.show()

É possível perceber que demorou um pouco para ocorrer a formação de um padrão na duração média dos episódios das séries, mas que por volta dos anos 60 esse padrão se definiu, entre os 40 e 60 minutos. Vimos que existem muitas séries com episódios curtos de 30 minutos, mas as as séries com mais de 50 minutos são mais numerosas. O início do gráfico, principalmente entre 1924 e 1934, apresenta uma variação bem grande de tempo médio de episódio. 

Vamos ver as séries com os menores e maiores tempos por episódio:

In [ ]:
series_df[series_df['episodeRuntimeSeries'] > 0][['episodeRuntimeSeries', 'name', 'startSeriesYear']].sort_values(by = ['episodeRuntimeSeries', 'startSeriesYear']).head(10)

Existem muitas séries com duração de 1 minuto! Exemplo de série de duração de 1 minuto: Minutos do Bicentenário foi uma série de curtos segmentos educacionais da televisão americana comemorando o bicentenário da Revolução Americana. Snipets era uma série de curtas-metragens de 30 segundos a um minuto para crianças.


In [ ]:
series_df[series_df['episodeRuntimeSeries'] > 0][['episodeRuntimeSeries', 'name', 'startSeriesYear']].sort_values(by = ['episodeRuntimeSeries', 'startSeriesYear'], ascending=False).head(10)

As séries de episódios mais longos são falhas de interpretação do campo. Por exemplo, no primeiro caso o tempo de execução está 8.400 min, mas esse é referente aos 280 episódios da série e não a cada um. 
Consideramos a possibilidade de erro de interpretação do campo no momento da junção das bases do TMDB e IMDB. No entanto analisando os casos acima verificamos que nas bases de origem mesmo temos o erro de interpretação. Por vezes o tempo cadastrado se refere a todos os episódios e por vezes se refere a um tempo por episódio. 

Vamos estimar o tamanho desse problema:

In [ ]:
# linhas em que o tempo por episódio está maior que 240 minutos (4 horas de duração)
problems = series_df.apply(lambda x: True if x['episodeRuntimeSeries'] > 240 else False , axis=1)
 
#print(series_df[problems][['episodeRuntimeSeries', 'number_of_episodes', 'name']])    
# Count number of True in series
numOfRows = len(problems[problems == True].index)
 
print('Number of Rows in dataframe in which episodeRuntimeSeries > 240 : ', numOfRows)

# Count number of True in series
numOfRows = len(problems[problems == False].index)
print('Number of Rows in dataframe in which episodeRuntimeSeries <= 240 : ', numOfRows)

Com a estimativa realizada, vemos que esse problema ocorre em menos de 1% dos registros. 

#### Línguas

Vamos analisar o número de línguas faladas em uma série. Vamos aproveitar para tratar o atributo languages da seguinte forma - ele será subtituído pelo número de línguas que apresenta, evitando assim os valores vazios:

In [ ]:
series_df['languages'] = series_df['languages'].fillna('[]').apply(ast.literal_eval).apply(lambda x: len(x) if isinstance(x, list) else np.nan)
series_df['languages'].value_counts()



A maioria das séries com campo preenchido apresenta língua única (46409). A grande maioria não têm essa informação (193845). A série com maior número de línguas tem 17 línguas registradas.

In [ ]:
series_df[series_df['languages'] >= 6][['name', 'startSeriesYear', 'languages']].sort_values('languages', ascending=False)

A série com mais línguas registradas na base, 17 línguas, é "Total Drama: Revenge of the Island" que é a quarta temporada da franquia "Total Drama". É um desenho que faz uma paródia de reality shows.

A segunda na lista é "Cold War": uma série de documentários televisivos de 24 episódios sobre a Guerra Fria que foi ao ar em 1998. Apresenta várias entrevistas e cenas dos eventos que moldaram as relações tensas entre a União Soviética e os Estados Unidos. O caráter global do evento tratado e o fato de ser um documentário repleto de entrevistas justifica a diversidade linguística da série.



### Séries ao longo dos anos
Vamos analisar o lançamento de séries ao longo dos anos:

In [ ]:
year_count = series_df.groupby('startSeriesYear')['name'].count()
plt.figure(figsize=(15,3))
year_count.plot()

O gráfico de distribuição de números de séries produzidas ao longo do tempo mostra a forte tendência de aumento na produção de séries.

In [ ]:
series_df[['name', 'startSeriesYear']].sort_values('startSeriesYear').head(10)

Analisando a lista de séries mais antigas (de 1897 a 1916) vemos programas diários, shows do Charlie Chaplin e um seriado francês (Judex).

### Verificação de colunas categóricas

Analisaremos os atributos status, type, titleType e isAdult

In [ ]:
series_df['status'].value_counts()

In [ ]:
series_df['type'].value_counts()

In [ ]:
series_df['titleType'].value_counts()

In [ ]:
series_df['isAdult'].value_counts()

Observando a distribuição das séries em relação a esses atributos, é possível notar que:

- Seguindo raciocínio lógico, temos mais séries finalizadas do que em andamento.
- Em relação ao tipo temos uma forte presença das séries roteirizadas, que contam uma história ao longo de temporadas que vão sendo desenvolvidas ao longo do tempo. Em seguida, séries no formato mini-série, com início e fim pré-determinados. Depois séries de "reality" - shows de tv que têm suas temporadas, mas que não contam histórias. Temos também documentários, talk shows, news e video.

As colunas isAdult e titleType, provenientes do IMDB, apresentam 79.40% de preenchimento. Já type e status, provenientes do TMDB, apresentam 34.32% de preenchimento. Para futuras análises em que aplicaremos classificadores, vamos:
preencher os valores nulos das colunas type, titleType e status com o valor 'unknown'
excluir o atributo isAdult, pois este se mostra irrelevante para a análise em questão.


In [ ]:
series_df['titleType'] = series_df['titleType'].fillna('unknown')
series_df['status'] = series_df['status'].fillna('unknown')
series_df['type'] = series_df['type'].fillna('unknown')
series_df = series_df.drop('isAdult', axis=1)


missingValuesSnapshot(series_df)


### Língua original

Vamos verificar a diversidade de línguas das séries na base do TMDB e a preponderância do inglês. 


In [ ]:
series_df['original_language'].drop_duplicates().shape[0]

In [ ]:
lang_df = pd.DataFrame(series_df['original_language'].value_counts())
lang_df['language'] = lang_df.index
lang_df.columns = ['number', 'language']
lang_df.head()

Existem 88 línguas de origem diferentes no dataset. Como esperado, o inglês é fortemente dominante. O japonês assume a segunda posição.

Gráfico abaixo excluindo o inglês para poder perceber melhor a diferença de incidência entre as outras linguagens.

In [ ]:
plt.figure(figsize=(12,5))
sns.barplot(x='language', y='number', data=lang_df.iloc[1:15])
plt.show()

O número de séries japonesas realmente é preponderante (desconsiderando o inglês). Vemos dentre alguns exemplos dessas séries japonesas, séries de desenho e super-heróis:

In [ ]:
series_df[series_df['original_language']== 'ja'][['name', 'startSeriesYear']].sort_values('startSeriesYear').head(10)

### Análise das bases de premiações

As bases do EMMY e do Golden Globe foram tratadas de forma a apresentar a mesma estrutura com os campos: year, category, winner e seriesName.

#### Número de categorias ao longo dos anos
Ambas as premiações apresentam diversas categorias de premiação. Vamos verificar como a quantidade de categorias diferentes variou ao longo dos anos. Lembrando que as bases aqui trabalhadas podem apresentar categorias além das destinadas somente à séries:

In [ ]:
def showCategoriesOverTheYears(df):
    df  = df.groupby('year').category.nunique().to_frame(name = 'num_categories').reset_index()
    plt.figure(figsize=(20,7))
    chart = sns.barplot(x='year', y='num_categories', data=df)
    chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
    plt.show()

showCategoriesOverTheYears(emmy_df)


In [ ]:
showCategoriesOverTheYears(gg_df)

Vemos que o Emmy apresenta um número máximo de categorias bem maior, quase 4 vezes maior, que o Golden Globe. Além disso:
- para o dataset do EMMY: as categorias aparecem desde o ano de 1949 até o ano de 2017 e, apesar de variarem consideravelmente em alguns anos, apresentam tendência de crescimento. 1965 é um ano que se destaca com o menor número de categorias. Pesquisando, encontramos a informação que nesse ano a estrutura da cerimônia bem diferente dos anos anteriores. As categorias foram simplificadas para que houvesse apenas quatro categorias principais (o ano anterior tinha 20 categorias principais). Como resultado, apenas cinco shows ganharam um prêmio. O novo formato foi descartado, e no ano seguinte o Emmy voltou ao formato tradicional.
- para o dataset do Golden Globe: as categorias aparecem no intervalo desde o ano de 1944 até o ano de 2011. Regra geral a tendência também é de crescimento do número de categorias mas com intervalos de estabilização.


#### Obtendo número de nomeações e prêmios por série

In [ ]:
#Agrupar por nome da série para obter em um mesmo dataframe o número de nomeações e de premiações de uma série 
#no EMMY e no GG.

def groupBySeriesNomineesAndPrizes(df):
    # agrupa pelo nome da série
    grp = df.groupby('seriesName', sort=False)
    prizes_df = pd.DataFrame({'prizes': grp['winner'].sum(),
                               'nominees':  grp.size()}).reset_index()
    print("Mais premiados: ")
    print(prizes_df[prizes_df.prizes == prizes_df.prizes.max()])
    print("Mais nomeados: ")
    print(prizes_df[prizes_df.nominees == prizes_df.nominees.max()])
    return prizes_df

prizes_nominees_emmy = groupBySeriesNomineesAndPrizes(emmy_df)
prizes_nominees_gg = groupBySeriesNomineesAndPrizes(gg_df)



Vamos agora fazer a junção dos datasets de premiação:

In [ ]:
# o merge já foi gerado no notebook 5
prizes_nominees_path = '../bases_tratadas/prizes_nominees_df.csv';
prizes_nominees_df =pd.read_csv(prizes_nominees_path)

print("Mais premiados em ambas as premiações: ")
print(prizes_nominees_df[prizes_nominees_df.total_prizes == prizes_nominees_df.total_prizes.max()])
print("Mais nomeados em ambos as nomeações: ")
print(prizes_nominees_df[prizes_nominees_df.total_nominees == prizes_nominees_df.total_nominees.max()])



## Junção de dataset agrupado de premiações por série com dataset de séries

Com essa junção estamos restringindo os registros de nomeação e de premiações a séries que existam na base construída a partir do TMDB e IMDB.

In [ ]:
series_prizes_nominees_df =  pd.merge(series_df, prizes_nominees_df, how='left', left_on=['name'],
                                    right_on =['seriesName'])


print("Número de séries com nomeação ou prêmio - data frames EMMY e GG: " + str(len(prizes_nominees_df)))
print("Número de séries no arquivo de séries: " + str(len(series_df)))

print("Número de séries com nomeação ou prêmio no dataframe final - JOINED: ")
print(str(len(series_prizes_nominees_df.loc[series_prizes_nominees_df['total_nominees'].notnull()])))

Como a junção das bases foi feita com base no valor exato do nome da série, podemos ter perdido algumas combinações de registros. No entanto vamos optar por trabalhar com esses registros de cuja combinação temos maior certeza. Vamos salvar a base para posterior trabalho de classificação:

In [ ]:
series_prizes_nominees_df.to_csv('../bases_tratadas/series_prizes_nominees_df.csv')


In [ ]:
missingValuesSnapshot(series_prizes_nominees_df)

### Análise dos nomes das séries nas diferentes bases

Será analisada a nuvem de palavras que ilustra a recorrência de termos nos títulos das séries em geral e das séries que tiveram nomeações, no EMMY e no GOLDEN GLOBE.

In [ ]:
def wordCloudOfAColumn(df, columnName):
    df[columnName] = df[columnName].astype('str')
    corpus = ' '.join(df[columnName])
    title_wordcloud = WordCloud(stopwords=STOPWORDS, background_color='white', height=2000, width=4000).generate(corpus)
    plt.figure(figsize=(16,8))
    plt.imshow(title_wordcloud)
    plt.axis('off')
    plt.show()


#Título geral
wordCloudOfAColumn(series_prizes_nominees_df, 'name')


In [ ]:
#Título No EMMY ou Golden Globe que não seja nulo
df_tmp = series_prizes_nominees_df[pd.notnull(series_prizes_nominees_df.seriesName)]
#print(df_tmp)
wordCloudOfAColumn(df_tmp, 'seriesName')



A nuvem de palavras de séries em geral mostra termos bem recorrentes nos nomes de programas de tv (TV, Show, Live, Life, Love, New, World, etc). Já nuvem de palavras de séries que foram pelo menos nomeadas ao Emmy ou ao Golden Globe também apresenta algumas dessas palavras em destaque (Show, World) mas apresenta forte recorrência nas palavras (American, Family, Special, War). Tais palavras estão presentes em títulos de séries nomeados nos eventos Emmy e Golden Globe. Nessa nuvem também é possível ler títulos inteiros de séries, como “Sherlock Holmes”.